# Scraping item collection/case series, and release date

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import requests
import warnings
warnings.filterwarnings('ignore')

In [48]:
collections = requests.get('https://www.csgodatabase.com/collections/')
cases =  requests.get('https://www.csgodatabase.com/cases/')

In [ ]:
bscollections = BeautifulSoup(collections.content)
bscase = BeautifulSoup(cases.text)

## Putting collection release date, names, and links into a dict. I will use the links to get the items that were released in that collection 

In [58]:
collection_dict = []
for x in a:
    current_dict = {}
    current_dict['collection'] = x.div.text
    current_dict['link'] = x.a['href']
    current_dict['release_date'] = x.find(attrs={'class':"introduced-skinbox"}).text
    collection_dict.append(current_dict)

In [59]:
collection_dict

[{'collection': 'The Alpha Collection',
  'link': 'https://www.csgodatabase.com/collections/the-alpha-collection/',
  'release_date': '19 September 2013'},
 {'collection': 'The Assault Collection',
  'link': 'https://www.csgodatabase.com/collections/the-assault-collection/',
  'release_date': '14 August 2013'},
 {'collection': 'The Aztec Collection',
  'link': 'https://www.csgodatabase.com/collections/the-aztec-collection/',
  'release_date': '14 August 2013'},
 {'collection': 'The Baggage Collection',
  'link': 'https://www.csgodatabase.com/collections/the-baggage-collection/',
  'release_date': '1 July 2014'},
 {'collection': 'The Bank Collection',
  'link': 'https://www.csgodatabase.com/collections/the-bank-collection/',
  'release_date': '1 May 2014'},
 {'collection': 'The Cache Collection',
  'link': 'https://www.csgodatabase.com/collections/the-cache-collection/',
  'release_date': '8 August 2014'},
 {'collection': 'The Chop Shop Collection',
  'link': 'https://www.csgodatabase.c

## Putting case release dates, names, and links into a dict. I will use the links to get the items that were released in that case. The html for this pages was harder to work with for some reason 

### Confirming lengths:

In [155]:
len(b.find_all('a'))

26

In [156]:
len(b.find_all(attrs={'class':'ConsumerSkinBox'}))

26

In [157]:
len(b.find_all(attrs={'class':"introduced-skinbox"}))

26

In [154]:
b = bscase.find(attrs={'class': 'entry-content'})
cases_dict = []
for x in range(len(b.find_all('a'))):
    cases_dict.append({})
for i, x in enumerate(b.find_all(attrs={'class':'ConsumerSkinBox'})):
    cases_dict[i]['case'] = x.text
for i, x in enumerate(b.find_all('a')):
    cases_dict[i]['link'] = x['href']
for i, x in enumerate(b.find_all(attrs={'class':"introduced-skinbox"})):
    cases_dict[i]['release_date'] = x.text

In [158]:
cases_dict

[{'case': 'Clutch Case',
  'link': 'https://www.csgodatabase.com/cases/clutch-case/',
  'release_date': '15 February 2018'},
 {'case': 'Spectrum 2 Case',
  'link': 'https://www.csgodatabase.com/cases/spectrum-2-case/',
  'release_date': '14 September 2017'},
 {'case': 'Operation Hydra Case',
  'link': 'https://www.csgodatabase.com/cases/operation-hydra-case/',
  'release_date': '23 May 2017'},
 {'case': 'Spectrum Case',
  'link': 'https://www.csgodatabase.com/cases/spectrum-case/',
  'release_date': '15 March 2017'},
 {'case': 'Glove Case',
  'link': 'https://www.csgodatabase.com/cases/glove-case/',
  'release_date': '28 November 2016'},
 {'case': 'Gamma 2 Case',
  'link': 'https://www.csgodatabase.com/cases/gamma-2-case/',
  'release_date': '18 August 2016'},
 {'case': 'Gamma Case',
  'link': 'https://www.csgodatabase.com/cases/gamma-case/',
  'release_date': '15 June 2016'},
 {'case': 'Chroma 3 Case',
  'link': 'https://www.csgodatabase.com/cases/chroma-3-case/',
  'release_date': '2

## Now I need to scape every 'link' and get the item names associated with that case/collection

In [160]:
r = requests.get('https://www.csgodatabase.com/cases/csgo-weapon-case/')

In [ ]:
bs = BeautifulSoup(r.content)

In [ ]:
for i in range(len(collection_dict)):
    r = requests.get(collection_dict[i]['link'])
    bs = BeautifulSoup(r.content)
    temp_list = []
    for item in bs.find_all(attrs={'class':'skin-box-header'}):
        temp_list.append(item.text)
    collection_dict[i]['items'] = temp_list

In [ ]:
for i in range(len(cases_dict)):
    r = requests.get(cases_dict[i]['link'])
    bs = BeautifulSoup(r.content)
    temp_list = []
    for item in bs.find_all(attrs={'class':'skin-box-header'}):
        temp_list.append(item.text)
    cases_dict[i]['items'] = temp_list

## Turns out csgodatabase doesn't have all of the items for each case. I can still use the release dates, but I am switching to scraping csgostash.com

In [229]:
r = requests.get('https://csgostash.com/case/244/Horizon-Case')

In [230]:
bs = BeautifulSoup(r.content)

### This gets the links from the first two "small" dropdown menus, which are "cases" and "collections"

In [231]:
links = bs.find_all('ul', attrs={'class':'dropdown-menu navbar-dropdown-small'})[:2]

### Creating list of dicts of case/collection name and links

In [232]:
case_dict2 = []
for link in links[0].find_all('a'):
    case_dict2.append({'case':link.text, 'link':link['href']})
case_dict2.pop()
case_dict2.pop()
case_dict2.pop()

{'case': 'All Skin Cases',
 'link': 'https://csgostash.com/containers/skin-cases'}

In [233]:
collection_dict2 = []
for link in links[1].find_all('a'):
    collection_dict2.append({'case':link.text, 'link':link['href']})

### From each case/collection link, getting all of the items and removing the extra h3 "Recently Added Items"

In [234]:
for case in range(len(case_dict2)):
    r = requests.get(case_dict2[case]['link'])
    bs = BeautifulSoup(r.content)
    items = [x.text for x in bs.find_all('h3')]
    try:
        items.remove('Recently Added Items')
    except:
        pass
    case_dict2[case]['items'] = items

In [235]:
for collection in range(len(collection_dict2)):
    r = requests.get(collection_dict2[collection]['link'])
    bs = BeautifulSoup(r.content)
    items = [x.text for x in bs.find_all('h3')]
    try:
        items.remove('Recently Added Items')
    except:
        pass
    collection_dict2[collection]['items'] = items

### Making a new dictionary of case/collection name : list of items to make it easy to transfer to the other list of dictionaries

In [240]:
conversion_list_cases = {}
for x in case_dict2[1:]:
    conversion_list_cases[x['case']] = x['items']

In [241]:
conversion_list_collections = {}
for x in collection_dict2:
    conversion_list_collections[x['case']] = x['items']

### Transfer the real item list to the list of dictionaries with release dates

In [248]:
for case in cases_dict:
    case['items'] = conversion_list_cases[case['case']]

In [254]:
for collection in collection_dict:
    collection['items'] = conversion_list_collections[collection['collection'][4:]]

## Transfer to MongoDB

In [1]:
from pymongo import MongoClient
client = MongoClient()
db = client['steam_capstone']
collection = db['item_release_dates']

In [258]:
# collection.insert_many(cases_dict)
# collection.insert_many(collection_dict)

## Bring release dates into the pickle

In [3]:
with open('data/all_apps_df.pkl', 'rb') as f:
    df = pickle.load(f)

In [4]:
csdf = df[df.app == 730]
csdf['release_date'] = np.nan

In [5]:
releases = list(collection.find())

#### Seems like this should work, just going to take a while?

In [6]:
for entry in releases:
    for item in entry['items']:
        mask = [item + ' (' in x for x in csdf['item_name']]
        csdf.loc[mask, 'release_date'] = entry['release_date']

In [7]:
csdf = csdf.reset_index(drop=True)

In [9]:
with open('csgo_df.pkl', 'wb') as f:
    pickle.dump(csdf, f)